<a href="https://colab.research.google.com/github/AllanKamimura/AI/blob/master/buzzleixo/buzzleixo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Buzzleixo
is a CNN for image classification, it takes an image input of either a BuzzLightyear or a Vovó Juju and returns a binary classification.

### Inicializador

In [2]:
#importa as bibliotecas ai
import os
import zipfile
import datetime
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from google.colab import files
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
#baixa os arquivos do dataset
!wget --no-check-certificate \
    https://github.com/AllanKamimura/AI/blob/master/buzzleixo/train.zip?raw=true \
    -O /tmp/Train.zip

--2020-12-25 15:56:05--  https://github.com/AllanKamimura/AI/blob/master/buzzleixo/train.zip?raw=true
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/AllanKamimura/AI/raw/master/buzzleixo/train.zip [following]
--2020-12-25 15:56:05--  https://github.com/AllanKamimura/AI/raw/master/buzzleixo/train.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/AllanKamimura/AI/master/buzzleixo/train.zip [following]
--2020-12-25 15:56:05--  https://raw.githubusercontent.com/AllanKamimura/AI/master/buzzleixo/train.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awai

### Estrutura dos dados
```
── train
    ├── Buzz
    │   ├── Buzz1.jpg
    │   ├── Buzz2.jpg
    │   └──  ...
    └── Juju
        ├── Juju1.jpg
        ├── Juju2.jpg
        └──  ...
```

In [4]:
main_path = "/tmp"
local_zip = '/tmp/Train.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall(main_path)
zip_ref.close()

# Directory with our training buzz pictures
train_buzz_dir = os.path.join(main_path, "train/Buzz")

# Directory with our training juju pictures
train_juju_dir = os.path.join(main_path, "train/Juju")

train_buzz_names = os.listdir(train_buzz_dir)
print(train_buzz_names[:10])

train_juju_names = os.listdir(train_juju_dir)
print(train_juju_names[:10])

print('total training buzz images:', len(os.listdir(train_buzz_dir)))
print('total training juju images:', len(os.listdir(train_juju_dir)))

['BLTitle.jpg', 'Screenshot_11.jpg', 'D_NQ_NP_985131-MLB31840915989_082019-V.jpg', 'Screenshot_12.jpg', 'final_5cae268d4dc5770013bcd43f_709784.jpg', '7899871606458Amp.jpg', 'maxresdefault.jpg', 'images.jpg', '1vmrbh.jpg', 'buzz-lightyear.jpg']
['Screenshot_11.jpg', 'Screenshot_1611122.jpg', '21276462a6f63c2244376bbdff819e83.jpg', 'tumblr_61b805199a3ebd6db51989787dd84cec_893e86ea_400.jpg', 'maxresdefault.jpg', 'images.jpg', 'tumblr_pms7pwV4sB1xgc3e1o4_250.jpg', '25ce22d505d22097319cedacde8893d8.jpg', 'images277.jpg', '7aa5da8bf992433d0c7c42e782882a0f.jpg']
total training buzz images: 50
total training juju images: 50


In [5]:
# All images will be rescaled by 1./255 + data augmentation

train_datagen = ImageDataGenerator(
    rescale = 1/255,
    rotation_range = 45,
    width_shift_range = 0.4,
    height_shift_range = 0.4,
    shear_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True,
    zoom_range = 0.5,
    fill_mode = 'nearest')


# Flow training images in batches  using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        os.path.join(main_path,'train'),  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 300x300
        batch_size= 10,
        shuffle = True,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode = 'binary')

Found 100 images belonging to 2 classes.


### Rede

In [6]:
nodes = 32 

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D( nodes , (3,3), activation = "relu", input_shape = (300,300,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D( 2*nodes, (3,3), activation = "relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D( 4*nodes, (3,3), activation = "relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D( 8*nodes, (3,3), activation = "relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense( 10*nodes, activation = "relu"),
    tf.keras.layers.Dense(1, activation = "sigmoid")])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 256)       2

In [7]:
model.compile(
    loss = "binary_crossentropy", 
    optimizer = "RMSprop", 
    metrics = ["accuracy"])

In [8]:
callback = tf.keras.callbacks.EarlyStopping(monitor = "accuracy", patience = 10, restore_best_weights = True)
history = model.fit(
    train_generator,
    steps_per_epoch = 10,
    batch_size = 10,
    epochs = 100,
    verbose = 1,
    callbacks = [callback])

Epoch 1/100
10/10 [==============================] - 11s 291ms/step - loss: 11.1105 - accuracy: 0.4472
Epoch 2/100
10/10 [==============================] - 3s 324ms/step - loss: 0.7025 - accuracy: 0.5214
Epoch 3/100
10/10 [==============================] - 3s 321ms/step - loss: 0.6938 - accuracy: 0.5020
Epoch 4/100
10/10 [==============================] - 3s 297ms/step - loss: 0.7018 - accuracy: 0.5497
Epoch 5/100
10/10 [==============================] - 3s 316ms/step - loss: 0.6707 - accuracy: 0.5761
Epoch 6/100
10/10 [==============================] - 3s 310ms/step - loss: 0.6753 - accuracy: 0.5413
Epoch 7/100
10/10 [==============================] - 3s 296ms/step - loss: 0.7180 - accuracy: 0.5627
Epoch 8/100
10/10 [==============================] - 3s 297ms/step - loss: 0.7013 - accuracy: 0.5977
Epoch 9/100
10/10 [==============================] - 3s 318ms/step - loss: 1.2439 - accuracy: 0.5528
Epoch 10/100
10/10 [==============================] - 3s 314ms/step - loss: 1.3399 - accu

### Faça o upload da imagem e veja o resultado

In [ ]:
uploaded = files.upload() #botão de fazer upload

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn #salva as imagens uploudadas aqui
  img = image.load_img(path, target_size=(300, 300)) #usa o keras.preprocess para abrir a imagem
  x = image.img_to_array(img) # transforma a imagem em um array
  x = np.expand_dims(x, axis=0) #aumenta as dimensões do array (batch_size, altura, largura, canal)

  images = np.vstack([x])
  classes = model.predict(images, batch_size = 10)
  if classes[0] > 0.5:
    print(fn + " É UM ABACATE")
    img = image.load_img(os.path.join(main_path,'aleixo.jpg'))
    plt.imshow(img)
    plt.show()
  else:
    print(fn + " É UM BUZZLATIR")
    img = image.load_img(os.path.join(main_path,'buzz.jpg'))
    plt.imshow(img)
    plt.show()

### Salve seus pesos
obs os arquivos temporarios (/tmp/) serão deletados quando a sessão for encerrada

In [ ]:
#cria um savepoint com a hora num fuso horario perdido ai
def save():
    now = datetime.datetime.now() 
    now = "{}_{}_{}_{}_{}".format(now.year,now.month,now.day,now.hour,now.minute)
    model.save(os.path.join(main_path, now + ".h5"))
save()